In [1]:
from math import log

def createDataSet():
    dataSet = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

#计算给定数据集的信息熵
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    # numEntries
    labelCounts = {} #创建一个空的字典
    for featVec in dataSet:  #一个一个样本的遍历数据集
        currentLabel = featVec[-1]    #取第一个样本的最后一个属性
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0  #将样本的最后一个属性加入到字典中最为键
        labelCounts[currentLabel] += 1     #存储键的值
        shannonEnt = 0
        for key in labelCounts:
            prob = float(labelCounts[key])/numEntries
            shannonEnt -= prob*log(prob, 2)
    return shannonEnt    

IndentationError: unexpected indent (<ipython-input-1-900efbbc183a>, line 10)